In [1]:
from ib_insync import *
import pandas as pd
import time
import nasdaqdatalink
import sys
from datetime import datetime, timedelta
import numpy as np
from sklearn.linear_model import LinearRegression

sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.sharadar_utils.sharadar_constants import FAMA_INDUSTRY
nasdaqdatalink.ApiConfig.api_key = ''

In [2]:
universe = nasdaqdatalink.get_table('SHARADAR/TICKERS', table="SF1")
df = pd.DataFrame(universe)
universe_df = df[df['isdelisted'] == 'N']
universe_df.to_csv('universe.csv')

/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/nasdaqdatalink/get_table.py:38: UserWarning: To request more pages, please set paginate=True in your         nasdaqdatalink.get_table() call. For more information see our documentation:         https://github.com/Nasdaq/data-link-python/blob/main/FOR_ANALYSTS.md#things-to-note
  warnings.warn(Message.WARN_PAGE_LIMIT_EXCEEDED, UserWarning)


In [4]:
df_daily = nasdaqdatalink.get_table('SHARADAR/SEP', qopts={"columns":['date' ,'close']}, ticker="AAPL")


In [5]:
df_quarterly = nasdaqdatalink.get_table('SHARADAR/SF1', dimension="MRT", qopts={"columns":['calendardate' ,'pe']}, ticker="AAPL")


In [6]:
# Assume df_daily and df_quarterly are your daily and quarterly DataFrames

# Reverse and set index for df_daily
df_daily = df_daily.sort_values(by='date', ascending=False).set_index('date')

# Reverse and set index for df_quarterly
df_quarterly = df_quarterly.sort_values(by='calendardate', ascending=False).set_index('calendardate')

# Merge DataFrames on common dates
merged_df = df_daily.merge(df_quarterly, left_index=True, right_index=True, how='inner')

merged_df = merged_df[::-1]

merged_df




,close,pe
1997-12-31,0.117,-1.939
1998-03-31,0.245,-31.098
1998-06-30,0.256,89.315
1998-09-30,0.341,16.885
1998-12-31,0.365,12.831
...,...,...
2022-03-31,174.610,27.972
2022-06-30,136.720,23.012
2022-09-30,138.200,24.223
2023-03-31,164.900,27.661


In [7]:
# Calculate correlation between the two value columns
correlation = merged_df['pe'].corr(merged_df['close'])

correlation


0.06979000025183062

In [12]:
x = merged_df['pe']
y = merged_df['close']

X = x.values.reshape(-1, 1)

# Create a LinearRegression model
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Access the intercept and coefficients
intercept = model.intercept_
coefficient = model.coef_

coefficient

array([0.03458403])